<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/finished_universal_trading_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
# ✅ Universal Trading Pipeline with Google Drive Integration and Enhancements
import pandas as pd
import numpy as np
import yfinance as yf
import time, os, tensorflow as tf
import matplotlib.pyplot as plt
import zipfile
from math import ceil
from shutil import copyfile
from datetime import datetime
from google.colab import drive
import json

# ✅ Mount Google Drive and Create Save Folder
drive.mount('/content/drive')
strategy_name = 'buy_volume_macd_rsi_v1'
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
save_dir = f"/content/drive/My Drive/universal_outputs/{strategy_name}"
os.makedirs(save_dir, exist_ok=True)
print(f"Save directory is ready: {save_dir}")

# ✅ TensorFlow GPU Memory Configuration
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory issue: {e}")

os.environ['CUDA_HOME'] = '/usr/local/cuda-11.8'
os.environ['PATH'] += ':/usr/local/cuda-11.8/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-11.8/lib64'

# ✅ Ticker List and CONFIG
ticker_list = [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]

CONFIG = {
    'symbols': [],
    'period': '720d',
    'interval': '1h',
    'target': 'Target',
    'sharpe_threshold': 1.5,
    'return_threshold': 1.25,
    'strategy_name': strategy_name
}

# ✅ Download Function
def download_stock_data(ticker, period="720d", interval="1h", max_retries=5):
    for attempt in range(1, max_retries + 1):
        try:
            print(f"Attempt {attempt}: Downloading {ticker}...")
            df = yf.download(ticker, period=period, interval=interval)
            if not df.empty:
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                return df
            raise ValueError("Empty data")
        except Exception as e:
            print(f"Error: {e}. Retrying...")
            time.sleep(attempt * 5)
    print(f"Failed to download {ticker}")
    return None

# ✅ Feature Engineering
def compute_enhanced_features(df):
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.loc[:, ~df.columns.duplicated()]

    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['STD_20'] = df['Close'].rolling(20).std()
    df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
    df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']
    df['Lowest_Low'] = df['Low'].rolling(14).min()
    df['Highest_High'] = df['High'].rolling(14).max()
    denom = (df['Highest_High'] - df['Lowest_Low']).replace(0, np.nan)
    df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / denom) * 100
    df['ROC'] = df['Close'].pct_change(10)
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (typical_price - typical_price.rolling(20).mean()) / (0.015 * typical_price.rolling(20).std())
    df['PROC'] = ((df['Close'] - df['Close'].shift(12)) / df['Close'].shift(12)) * 100
    df['Rolling_Mean_50'] = df['Close'].rolling(50).mean()
    df['Expanding_Mean'] = df['Close'].expanding().mean()
    df['EMA_10'] = df['Close'].ewm(span=10).mean()
    df['EMA_50'] = df['Close'].ewm(span=50).mean()
    df['MACD_Line'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9).mean()
    df['MACD_Hist'] = df['MACD_Line'] - df['MACD_Signal']
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = -delta.where(delta < 0, 0).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['True_Range'] = df[['High', 'Low', 'Close']].apply(lambda x: max(x[0] - x[1], abs(x[0] - x[2]), abs(x[1] - x[2])), axis=1)
    df['ATR'] = df['True_Range'].rolling(14).mean()
    df['+DM'] = np.where((df['High'].diff() > df['Low'].diff()) & (df['High'].diff() > 0), df['High'].diff(), 0)
    df['-DM'] = np.where((df['Low'].diff() > df['High'].diff()) & (df['Low'].diff() > 0), df['Low'].diff(), 0)
    df['+DI'] = 100 * df['+DM'].rolling(14).mean() / df['ATR']
    df['-DI'] = 100 * df['-DM'].rolling(14).mean() / df['ATR']
    df['ADX'] = abs(df['+DI'] - df['-DI']).rolling(14).mean()
    df['Volume_Avg'] = df['Volume'].rolling(20).mean()
    df['Volume_Change'] = df['Volume'].pct_change()
    df['Volume_Change_MA'] = df['Volume_Change'].rolling(10).mean()
    df['Volume_Change_Ratio'] = df['Volume_Change'] / df['Volume_Change'].shift(1)
    df['Relative_Volume'] = df['Volume'] / df['Volume_Avg']
    df['Trailing_Stop'] = np.minimum(df['Close'] * 0.985, df['Close'] - (df['ATR'] * 0.3))
    df['Buy_Signal'] = np.where((df['RSI'] < 60) & (df['EMA_10'] > df['EMA_50']) &
                                ((df['MACD_Line'] > df['MACD_Signal']) | (df['MACD_Line'].diff() > 0)) &
                                (df['Volume'] > (0.4 * df['Volume_Avg'])) & (df['ADX'] > 18), 1, 0)
    df['Sell_Signal'] = np.where(((df['EMA_10'] < df['EMA_50']) & (df['RSI'] > 60)) |
                                 ((df['MACD_Line'] < df['MACD_Signal']) & (df['RSI'] > 65)) |
                                 (df['Close'] < df['Trailing_Stop']) |
                                 ((df['Volume'] > 0.5 * df['Volume_Avg']) & (df['ADX'] > 20)), 1, 0)
    df['Sell_Signal_Debug'] = np.where(((df['MACD_Hist'] < 0.5) | (df['MACD_Line'] < df['MACD_Signal'])) &
                                       (df['RSI'] < 55) & (df['ADX'] > 15) &
                                       ((df['Close'] < df['Trailing_Stop']) | (df['EMA_10'] < df['EMA_50'])) &
                                       (df['Volume'] > 0.5 * df['Volume_Avg']), 1, 0)
    df['Future_Close'] = df['Close'].shift(-10)
    df['Volatility'] = df['Close'].pct_change().rolling(window=20).std()

    df['Return'] = (df['Future_Close'] - df['Close']) / df['Close']
    df['Target'] = np.select([df['Return'] > 0.02, df['Return'] < -0.02], [1, -1], default=0)
    df['Multi_Class_Target'] = df['Target']
    df['Hour'] = pd.to_datetime(df['Datetime']).dt.hour
    df['DayOfWeek'] = pd.to_datetime(df['Datetime']).dt.dayofweek
    df['Session'] = np.where((df['Hour'] >= 9) & (df['Hour'] <= 16), 'Regular',
                             np.where((df['Hour'] < 9), 'Pre-market', 'After-hours'))
    df['MACD_Crossover'] = np.where(df['MACD_Line'] > df['MACD_Signal'], 1, 0)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    df.dropna(inplace=True)
    return df

# ✅ Backtest & Metrics
def simple_backtest(df, target_col='Target', initial_cash=10000, position_size=1.0):
    df = df.copy()
    df['Return'] = df['Close'].pct_change()
    df['Strategy_Return'] = df[target_col].shift(1) * df['Return'] * position_size
    df['Equity_Curve'] = initial_cash * (1 + df['Strategy_Return']).cumprod()
    return df

def evaluate_performance_metrics(df):
    df = df.copy()
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df.set_index('Datetime', inplace=True)
    daily_returns = df['Equity_Curve'].pct_change().dropna()
    excess_returns = daily_returns
    sharpe = np.sqrt(252) * excess_returns.mean() / excess_returns.std()
    trades = df['Strategy_Return'][df['Strategy_Return'] != 0]
    win_loss = trades[trades > 0].count() / trades[trades < 0].count() if trades[trades < 0].count() > 0 else np.inf
    drawdown = (df['Equity_Curve'] - df['Equity_Curve'].cummax()) / df['Equity_Curve'].cummax()
    cagr = (df['Equity_Curve'].iloc[-1] / df['Equity_Curve'].iloc[0]) ** (1 / ((df.index[-1] - df.index[0]).days / 365.25)) - 1
    df.reset_index(inplace=True)
    return {'Sharpe Ratio': sharpe, 'Win/Loss Ratio': win_loss, 'Max Drawdown': drawdown.min(), 'CAGR': cagr}

# ✅ Run Pipeline
def run_universal_framework(config):
    all_results, all_metrics, filtered = {}, {}, []
    for symbol in config['symbols']:
        print(f"Processing {symbol}")
        df = download_stock_data(symbol, config['period'], config['interval'])
        if df is None: continue
        df = compute_enhanced_features(df)
        df = simple_backtest(df, config['target'])
        metrics = evaluate_performance_metrics(df)
        final_return = df['Equity_Curve'].iloc[-1]
        sharpe = metrics.get('Sharpe Ratio', 0)
        print(f"🔎 {symbol} Sharpe: {sharpe:.2f}, Final Return: {final_return:.2f}")
        if sharpe > config['sharpe_threshold'] or final_return > config['return_threshold']:
            all_results[symbol], all_metrics[symbol] = df, metrics
            filtered.append(symbol)
            df.to_csv(f"/content/{config['strategy_name']}_{symbol}_features.csv", index=False)
            pd.DataFrame.from_dict(metrics, orient='index', columns=['Value']).to_csv(f"/content/{config['strategy_name']}_{symbol}_metrics.csv")
    return all_results, all_metrics, filtered

# ✅ Chunk Execution
chunk_size = 10
combined_df = pd.DataFrame()
global_results, global_metrics, global_filtered = {}, {}, []

for i in range(0, len(ticker_list), chunk_size):
    CONFIG['symbols'] = ticker_list[i:i+chunk_size]
    print(f"Running chunk {i // chunk_size + 1}...")
    results, metrics, filtered = run_universal_framework(CONFIG)
    global_results.update(results)
    global_metrics.update(metrics)
    global_filtered.extend(filtered)

    for ticker in filtered:
        df = results[ticker]
        final_columns = [
            'Datetime', 'Open', 'High', 'Low', 'Close', 'Volume',
            'EMA_10', 'EMA_50', 'SMA_20', 'Rolling_Mean_50', 'Expanding_Mean',
            'RSI', 'MACD_Line', 'MACD_Signal', 'MACD_Hist',
            'ATR', 'Volatility', 'Volume_Avg', 'Volume_Change', 'Volume_Change_MA',
            'Volume_Change_Ratio', 'Relative_Volume', 'ADX', 'Trailing_Stop',
            'Buy_Signal', 'Sell_Signal', 'Sell_Signal_Debug',
            'Return', 'Target', 'Multi_Class_Target', 'Hour', 'DayOfWeek', 'Session',
            'MACD_Crossover', 'Future_Close']
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.dropna(inplace=True)
        df_labeled = df[final_columns]
        combined_df = pd.concat([combined_df, df_labeled], axis=0)
        path = f"{save_dir}/{CONFIG['strategy_name']}_{ticker}_labeled_dataset.csv"
        df_labeled.to_csv(path, index=False)
        print(f"Saved: {path}")

# ✅ Save Combined Dataset
combined_path = f"{save_dir}/combined_dataset_{timestamp}.csv"
combined_df.to_csv(combined_path, index=False)
print(f"Combined dataset saved: {combined_path}")

# ✅ Zip and Upload
with zipfile.ZipFile("/content/universal_strategy_outputs.zip", "w") as zipf:
    for ticker in global_filtered:
        zipf.write(f"/content/{CONFIG['strategy_name']}_{ticker}_features.csv")
        zipf.write(f"/content/{CONFIG['strategy_name']}_{ticker}_metrics.csv")
copyfile("/content/universal_strategy_outputs.zip", f"{save_dir}/universal_strategy_outputs.zip")
print("Zipped results uploaded to Google Drive")

# ✅ Log Filtered Tickers
pd.Series(global_filtered).to_csv(f"{save_dir}/filtered_tickers.csv", index=False)
print(f"Filtered tickers saved to: {save_dir}/filtered_tickers.csv")

# ✅ Strategy Summary Log (Enhanced with Filtered Ticker List)
summary_path = f"{save_dir}/summary_{timestamp}.txt"
with open(summary_path, "w") as f:
    f.write(f"Strategy: {strategy_name}\n")
    f.write(f"Total tickers evaluated: {len(ticker_list)}\n")
    f.write(f"Tickers passing filter: {len(global_filtered)}\n")
    f.write(f"Saved at: {timestamp}\n")
    f.write("Filtered Tickers:\n")
    f.write(", ".join(global_filtered) + "\n")

print(f" Summary log created at: {summary_path}")


# ✅ Save CONFIG as JSON
with open(f"{save_dir}/config.json", "w") as f:
    json.dump(CONFIG, f, indent=4)
print(f"CONFIG saved to: {save_dir}/config.json")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Save directory is ready: /content/drive/My Drive/universal_outputs/buy_volume_macd_rsi_v1
Running chunk 1...
Processing AAPL
Attempt 1: Downloading AAPL...


[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)"))


Error: Empty data. Retrying...
Attempt 2: Downloading AAPL...


[*********************100%***********************]  1 of 1 completed
<ipython-input-64-daab1caf773a>:106: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['True_Range'] = df[['High', 'Low', 'Close']].apply(lambda x: max(x[0] - x[1], abs(x[0] - x[2]), abs(x[1] - x[2])), axis=1)


🔎 AAPL Sharpe: 2.92, Final Return: 643819.01
Processing TSLA
Attempt 1: Downloading TSLA...


...

🔎 GE Sharpe: 3.21, Final Return: 1957206.00
Processing SBUX
Attempt 1: Downloading SBUX...


[*********************100%***********************]  1 of 1 completed
<ipython-input-64-daab1caf773a>:106: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['True_Range'] = df[['High', 'Low', 'Close']].apply(lambda x: max(x[0] - x[1], abs(x[0] - x[2]), abs(x[1] - x[2])), axis=1)


🔎 SBUX Sharpe: 2.11, Final Return: 471156.52
Processing ORCL
Attempt 1: Downloading ORCL...


[*********************100%***********************]  1 of 1 completed
<ipython-input-64-daab1caf773a>:106: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['True_Range'] = df[['High', 'Low', 'Close']].apply(lambda x: max(x[0] - x[1], abs(x[0] - x[2]), abs(x[1] - x[2])), axis=1)


🔎 ORCL Sharpe: 2.72, Final Return: 1985374.59
Saved: /content/drive/My Drive/universal_outputs/buy_volume_macd_rsi_v1/buy_volume_macd_rsi_v1_GE_labeled_dataset.csv
Saved: /content/drive/My Drive/universal_outputs/buy_volume_macd_rsi_v1/buy_volume_macd_rsi_v1_SBUX_labeled_dataset.csv
Saved: /content/drive/My Drive/universal_outputs/buy_volume_macd_rsi_v1/buy_volume_macd_rsi_v1_ORCL_labeled_dataset.csv
Combined dataset saved: /content/drive/My Drive/universal_outputs/buy_volume_macd_rsi_v1/combined_dataset_20250405_0113.csv
Zipped results uploaded to Google Drive
Filtered tickers saved to: /content/drive/My Drive/universal_outputs/buy_volume_macd_rsi_v1/filtered_tickers.csv
 Summary log created at: /content/drive/My Drive/universal_outputs/buy_volume_macd_rsi_v1/summary_20250405_0113.txt
CONFIG saved to: /content/drive/My Drive/universal_outputs/buy_volume_macd_rsi_v1/config.json
